
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="files/image/DD_07.png" alt="Databricks Learning" style="width: 1500px">
</div>

##  Feature Store


### 학습 목표 

- Data를 전처리하여 Feature Table 생성하고 Feature Store에 등록합니다.

**Feature Engineering**은 머신러닝 모델 개발 과정에서 중요한 전처리 단계입니다 \
이 단계에서는 원시 데이터를 모델이 이해할 수 있는 형식으로 변환하고, 모델의 성능을 최적화하기 위해 중요한 특징(Feature)을 선택하고 생성합니다

# Feature store 생성

In [0]:
# Feature Store를 사용하기 위한 Library를 Import 합니다.

from databricks.feature_engineering import FeatureEngineeringClient
from databricks.feature_store import FeatureLookup


In [0]:
# Model Building 을 위한 Library를 Import 합니다.

import pandas as pd

from pyspark.sql.functions import monotonically_increasing_id,expr,rand
import uuid


#sklearn 과 관련된 Library 와 Model 평가를 위한 Library를 Import 합니다. 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score



### 모델 학습용 데이터 전처리 과정

In [0]:
%py
import re

# TODO catalog_name 변경
catalog_name = 'edu2501'
username = spark.sql("SELECT current_user()").first()[0]
clean_username = re.sub("[^a-zA-Z0-9]", "_", username)

print(f'catalog_name = {catalog_name}')
print(f'username = {username}')
print(f'clean_username = {clean_username}')

spark.sql(f'set edu.catalog={catalog_name}')
spark.sql(f"set edu.username={username}")
spark.sql(f"set edu.clean_username={clean_username}")

In [0]:
%sql
-- Schema 생성
create schema if not exists ${edu.catalog}.${edu.clean_username};
use catalog ${edu.catalog};
use ${edu.catalog}.${edu.clean_username};

In [0]:
# 데이터 적재
raw_data = spark.read.load("/databricks-datasets/wine-quality/winequality-red.csv", format="csv", sep=";", header="true", inferSchema="true")

In [0]:
raw_data

In [0]:
display(raw_data)

In [0]:
# ID column을 추가하는 함수와, name을 변경하는 함수를 정의합니다.

def addIcolumn(dataframe, id_column_name):
    columns = dataframe.columns
    new_df = dataframe.withColumn(id_column_name, monotonically_increasing_id())
    return new_df[[id_column_name] + columns]

def renameColumn(df):
    renamed_df = df
    for column in df.columns:
        renamed_df = renamed_df.withColumnRenamed(column, column.replace(" ", "_"))
    return renamed_df
    



In [0]:
renamed_df = renameColumn(raw_data)
df = addIcolumn(renamed_df, "wine_id")

In [0]:
# quality를 예측 할 것이기 때문에 기존 데이터셋에서 해당 column을 drop 시켜줍니다.

feature_df = df.drop("quality")
display(feature_df)

In [0]:
#Feature Store 에 사용할 DB를 만들어 줍니다.
# spark.sql(f"USE CATALOG edu2501")
# spark.sql(f"CREATE SCHEMA IF NOT EXISTS wine_db")
# spark.sql(f"USE SCHEMA wine_db")


# 각 실행마다 고유의 UUID를 생성하여 Feature Store를 사용하기 위한 Table Name을 생성합니다.
table_name = f"wine_db_" + str(uuid.uuid4())[:6]
print(table_name)

### Feature Store에 Table 생성

In [0]:
# Feature Store 에 Table 생성에 필요한 Feature enginnering Client를 생성합니다.
fe = FeatureEngineeringClient()

### 생성한 Feature Engineering Client로 Feature store를 생성하고 관리 할 수 있습니다.

In [0]:
# Feature Store Client를 사용한 Feature Table 생성

fe.create_table(
    name=table_name,
    primary_keys=["wine_id"],
    df=feature_df,
    schema=feature_df.schema,
    description="Wine Quality Feature Table"
)

Databricks 왼쪽 툴바 **Features** 메뉴 에서 생성된 Feature Table을 확인 할 수 있습니다.

### Feature Store의 활용

In [0]:
# 실시간 측정값 시뮬레이션하고 추론을 위한 Dataset  생성.

inference_data_df = df.select("wine_id","quality",(10 * rand()).alias("real_time_measurement"))
display(inference_data_df)

In [0]:
# Taining, Testing, input, output 데이터셋을 생성합니다.

def load_data(table_name, lookup_key):
    model_feature_lookups = [FeatureLookup(table_name, lookup_key=lookup_key)]

    training_set = training_set = fe.create_training_set(
        df=inference_data_df,
        feature_lookups=model_feature_lookups,
        label='quality',
        exclude_columns=['wine_id']
    )
    training_pd = training_set.load_df().toPandas()


    # train, test 데이터셋 생성합니다.
    X = training_pd.drop(["quality"], axis=1)
    y = training_pd["quality"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, training_set


In [0]:
X_train, X_test, y_train, y_test, training_set = load_data(table_name, "wine_id")
X_train.head()

## Model 생성

In [0]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

try: 
    client.delete_registered_model("wine_model") # 이미 모델이 생성되어 있으면 삭제
except:
    None

In [0]:
import mlflow

# mlflow autologging을 사용하지 않고 Feature Store를 사용하여 모델의 log를 기록합니다.
mlflow.sklearn.autolog(log_models=False)

def train_model(X_train, X_test, y_train, y_test,training_set,fe):
    with mlflow.start_run() as run:

        rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        mlflow.log_metric("test_mse", mean_squared_error(y_test, y_pred))
        mlflow.log_metric("test_r2", r2_score(y_test, y_pred))

        fe.log_model(
            model = rf,
            artifact_path = "wine_quality_prediction",
            flavor = mlflow.sklearn,
            training_set = training_set,
            registered_model_name = "wine_model"
        )

train_model(X_train, X_test, y_train, y_test,training_set,fe)
